In [1]:
import os
import glob
import re
import configparser
import pandas as pd
import numpy as np
import xlwings as xl
import xlsxwriter
import math




In [2]:
def inifile(file):

    global ini_data

    fpath=file
    
    config = configparser.ConfigParser()
    config.read(fpath)

    data = []
    for section in config.sections():
        for key in config[section]:
            value = config[section][key]
            data.append([section, key, value])

    ini_data = pd.DataFrame(data, columns=['Freq', 'Test', 'Value'])
    # print(ini_data)

    # ini_data=ini_data.loc[(ini_data['Test']=='calibrationtarget') | (ini_data['Test']=='GoldenPhase'),:]
    ini_data = ini_data.loc[(ini_data['Test'] == 'calibrationtarget') | (ini_data['Test'] == 'goldenphase'), :]
    # print(ini_data)

    split_data=ini_data['Freq'].str.replace('Name','Freq').str.split('=',expand=True)

    ini_data = ini_data.drop(columns='Freq',axis=1)
    split_data = split_data.drop(columns=0,axis=1)

    ini_data=pd.concat([ini_data,split_data],axis=1)
    ini_data=ini_data.rename(columns={1:'Freq'})

    # ini_data['Value'] = ini_data['Value'].replace('#', np.nan)
    # ini_data['Value']=ini_data['Value'].astype(float)
    ini_data = ini_data[ini_data['Value'] != '#'].reset_index(drop=True)
    ini_data['Value'] = ini_data['Value'].astype(float)
    
    # print(ini_data.at[2,'Value'].dtype)

    print(ini_data)

In [3]:
%run A601_Column_List.ipynb

          0   1      2      3      4      5      6      7      8      9
0      16.5  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
1      20.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
2      35.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
3      75.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
4      80.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
5     300.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
6     900.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
7    1000.0  94  100.0  106.0  112.0  118.0  124.0  127.0  130.0  132.0
8    1100.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
9    3000.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
10   8000.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
11  10000.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
[1000.0, 94.0, 100.0, 106.0, 112.0, 118.0, 124.0, 127.0, 130.0, 

In [4]:

# based on A601 generate the list, search every frequency point, enter the each foler to find the ini file, and extract all ini file value to generate the new ini file target file.

folder_path = os.path.dirname(fpath)



# file name including "MicrophoneCharacterization" string
regex = re.compile(".*MicrophoneCharacterization.*")

writer = pd.ExcelWriter('New_ini_Target.xlsx', engine='xlsxwriter')

# based on column list value, using glob module try to search the folder
for value in column_list:
    search_pattern = os.path.join(folder_path, f"*{value}*")
    folders = glob.glob(search_pattern)
    if len(folders) > 0:
        for folder in folders:
            print(f"Found folder for value {value}: {folder}")
            # search the ini file from each folder
            files = glob.glob(os.path.join(folder, "*.ini"))
            if len(files) > 0:
                for file in files:
                    # search the file naming including "MicrophoneCharacterization" string
                    match = regex.search(file)
                    if match:
                        inifile(file)
                        # write the data in  sheet 
                        sheetname=f'{value}'
                        ini_data.to_excel(writer, sheet_name=sheetname, index=False)
            else:       
                print(f"No files found in folder {folder}")
    else:
        print(f"No folders found for value {value}")

# 保存 Excel 文件
writer.save()



Found folder for value 94: C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data\Data_94dBSPL
                 Test  Value     Freq
0   calibrationtarget -39.59     16.5
1         goldenphase  23.25     16.5
2   calibrationtarget -39.17     20.0
3         goldenphase  20.18     20.0
4   calibrationtarget -38.47     35.0
5         goldenphase  12.91     35.0
6   calibrationtarget -38.09     75.0
7         goldenphase   6.37     75.0
8   calibrationtarget -38.09     80.0
9         goldenphase   5.98     80.0
10  calibrationtarget -37.99    300.0
11        goldenphase   1.81    300.0
12  calibrationtarget -38.02    900.0
13        goldenphase   1.31    900.0
14  calibrationtarget -38.00   1000.0
15        goldenphase   1.34   1000.0
16  calibrationtarget -38.01   1100.0
17        goldenphase   1.46   1100.0
18  calibrationtarget -37.89   3000.0
19        goldenphase   3.44   3000.0
20  calibrationtarget -37.25   8000.0
21        goldenphase  10.14   8000.0
22  ca

C:\Users\wanglinl\AppData\Local\Temp\ipykernel_2524\3864698079.py:38: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
